In [1]:
import numpy as np
from aeon.datasets import load_classification
from sklearn.model_selection import train_test_split
import pickle
import pandas as pd
from collections import Counter

from source.models import Models
from source.morph2 import Morph

import pycatch22
import tsfel

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import time
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
ECG_datasets = ['ECG200', 'EOGHorizontalSignal', 'EOGVerticalSignal', 'Epilepsy', 
        'CinCECGTorso', 
        'ECG5000', 'ECGFiveDays', 'StandWalkJump', 
        'TwoLeadECG','NerveDamage' , 'MedicalImages',
        'Colposcopy',
        'EyesOpenShut', 'ToeSegmentation1', 'Heartbeat',
         'EMOPain', 'HandMovementDirection']

**Is it mandatory to pass the sampling rate?**

No. However, please note that some spectral features rely on the sampling rate to be calculated correctly. Therefore, if you have access to the sampling rate, it is a good practice to pass the correct value. The default sampling rate value is set to 100 Hz. In case you do not have access to the sampling rate, you might refrain from using spectral features.


**Does TSFEL allow to extract features from multi-dimensional time series with variable lengths?**

Yes, it is possible, indeed. That’s actually one of the functionalities that weren’t adequately addressed by similar packages when we started the development of TSFEL. I recommend that the time series be stored in a data file and processed using the dataset_features_extractor. We are still updating the documentation of this functionality. In the meantime, you can read Section 2.2.1. Data ingestion and preprocessing of TSFEL publication which addresses that topic.


In [ ]:
def multivariate_featues(row, cfg): 
    feature_names = []  
    row_features = []  

    for var_idx, variate in enumerate(row):
        features = tsfel.time_series_features_extractor(cfg, variate, verbose=0)
        
        modified_columns = [
            col.replace('0_', f'{var_idx}_') if var_idx != 0 else col 
            for col in features.columns
        ]
        features.columns = modified_columns  

        row_features.append(features)
        feature_names.extend(modified_columns) 
    
    combined_row_features = pd.concat(row_features, axis=1)
    dataframe = pd.DataFrame(combined_row_features, columns=feature_names)
    return dataframe

    
all_results = {}
all_features = {}
results_array = np.empty((0, 11)) 

for df_name in ECG_datasets:
    try:
        # Load Dataset ===================================
        X, y = load_classification(df_name)
        le = LabelEncoder()
        y = le.fit_transform(y)
    except Exception as e:
        print(f'{df_name}: Dataset Not Available - {str(e)}')
        continue

    print("-" * 70)
    print("Dataset Name:", df_name)
    print("-" * 50)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Useful Information
    ts_length = X_train.shape[2]
    df_size = X_train.shape[0]
    n_classes = len(np.unique(y))  
    variates = X_train.shape[1]  
    class_counts = Counter(y_train)
    num_classes = len(class_counts)

    print('X_train Shape:', X_train.shape)
    print('X_test Shape:', X_test.shape)

    print('y_train Counts:', np.unique(y_train, return_counts=True))
    print('y_test Counts:', np.unique(y_test, return_counts=True))
    print("-" * 50)


    # Train and Evaluate Models ===================================
    start = time.time()
    lstm = Models('lstm', X_train, y_train)
    lstm.train_lstm()
    catch = Models('catch22', X_train, y_train)
    catch.train_catch22()
    rocket = Models('rocket', X_train, y_train)
    rocket.train_rocket()

    models = [lstm, catch, rocket] 
    acc = {}
    for m in models:
        pred, _ = m.predict(X_test)
        acc[m.model_name] = accuracy_score(pred, y_test)

    print(acc)
    print("-" * 50)


    class_results = {}
    features_results = {}
    # Loop Through each Class ===================================
    for c in np.unique(y):
        start_class = time.time()
        print(f'Processing Class: {c}')

        # Calculate class percentage
        class_perc = round(class_counts[c] / df_size, 3)

        # Compute Morphing ===================================
        morphing = Morph(X_test, y_test, c)
        morphing.get_DTWGlobalBorderline(X_test.shape[0])
        results = morphing.CalculateMorph(models)

        end_class = time.time()
        print(f'Total Class {c} run time: {end_class - start_class}')

        class_decoed = le.inverse_transform([c])[0]
        print(class_decoed)
        class_results[class_decoed] = results


        # Feature Extraction using TSFEL ===================================
        # normalize features difference 
        models_features = {} 
        feature_names = None
        cfg_file = tsfel.get_features_by_domain() 
        for m in models:
            differences = []
            data = results[m.model_name]
            for i in range(len(data['morphs'])):

                morph = data['morphs'][i]
                change_inx = data['change_indice'][i]
               
                source = morph[0]
                target = morph[change_inx]
                
                # Univariate Time Series ===================================
                if morph.shape[1] == 1:
                    source_features = tsfel.time_series_features_extractor(cfg_file, source[0], verbose=0)
                    target_features = tsfel.time_series_features_extractor(cfg_file, target[0], verbose=0)
                    #print(source.shape, target.shape)

                # Multivariate Time Series ===================================
                else:
                    source_features = multivariate_featues(source, cfg_file)
                    target_features = multivariate_featues(target, cfg_file)

                if feature_names is None:
                    feature_names = source_features.columns
                
                source_features = np.array(source_features)
                target_features = np.array(target_features)

                percentage_diff = (target_features - source_features) / (np.abs(source_features) + 1e-10)

                differences.append(percentage_diff)

            avg_diff = np.mean(differences, axis=0).flatten()            
                        
            if np.isnan(avg_diff[0]):
                print(f"avg_diff is None for {m.model_name} in {class_decoed}. No correct model predictions!")
                models_features[m.model_name] = None
                continue
            else:
                diffs = {}
                for i in range(len(avg_diff)):
                    diffs[feature_names[i]] = avg_diff[i]

                models_features[m.model_name]  = sorted(diffs.items(), key=lambda x: abs(x[1]), reverse=True)

                print("-" * 50)
                print('Top 5 Feature Changes:')
                for feature, diff in models_features[m.model_name][:5]:
                    print(f'{feature}: {diff}')

        features_results[class_decoed] = models_features
     
        # Append results to NumPy array
        for model in results.keys():
            data = results[model]['metrics']
            line = np.array([[df_name, df_size, variates, ts_length, n_classes, class_decoed, class_perc, model, data['mean'], data['std'], acc[model]]])
            results_array = np.vstack((results_array, line))
        

    # Save results for the current dataset
    file_name = f'results/pickles/{df_name}.pkl'
    with open(file_name, 'wb') as f:
        pickle.dump(class_results, f)

    # Save features results for the current dataset
    file_name = f'results/pickles/features_{df_name}.pkl'
    with open(file_name, 'wb') as f:
        pickle.dump(features_results, f)

    # Add results to all_results
    all_results[df_name] = class_results
    all_features[df_name] = features_results

    # Clean ups
    del models, lstm, catch, rocket, class_results, features_results, X_train, X_test, y_train, y_test

    # Convert NumPy array to Pandas DataFrame
    columns = ['dataset', 'df_size', 'n_variates','ts_length', 'n_classes', 'class', 'class_perc', 'model', 'mean', 'std', 'model_acc']
    dataframe = pd.DataFrame(results_array, columns=columns)
    # Save results to CSV
    dataframe.to_csv('results/final_results2.csv', index=False)

    # Save all results to a single pickle file
    with open('results/pickles/final_results2.pkl', 'wb') as f:
        pickle.dump(all_results, f)
    
    # Save all features to a single pickle file
    with open('results/pickles/final_features2.pkl', 'wb') as f:
        pickle.dump(all_features, f)

print('All results saved successfully!')


----------------------------------------------------------------------
Dataset Name: ECG200
--------------------------------------------------
X_train Shape: (160, 1, 96)
X_test Shape: (40, 1, 96)
y_train Counts: (array([0, 1], dtype=int64), array([ 54, 106], dtype=int64))
y_test Counts: (array([0, 1], dtype=int64), array([13, 27], dtype=int64))
--------------------------------------------------
{'lstm': 0.9, 'catch22': 0.8, 'rocket': 0.9}
--------------------------------------------------
Processing Class: 0


100%|██████████| 40/40 [00:38<00:00,  1.05it/s]


Total Class 0 run time: 49.559783935546875
-1
--------------------------------------------------
Top 5 Feature Changes:
0_MFCC_2: 249.5502612697744
0_MFCC_8: 4.298398846100614
0_Spectrogram mean coefficient_8.06Hz: 4.243280873998029
0_Spectrogram mean coefficient_9.68Hz: 3.634506481649972
0_MFCC_4: -3.42071573136745
--------------------------------------------------
Top 5 Feature Changes:
0_MFCC_4: -5.832480325779804
0_Spectrogram mean coefficient_8.06Hz: 4.437106468496177
0_Spectrogram mean coefficient_9.68Hz: 4.051397830528047
0_LPCC_0: 3.779842132551862
0_MFCC_10: -2.5718534070948285
--------------------------------------------------
Top 5 Feature Changes:
0_MFCC_2: 167.82828228709164
0_MFCC_4: -3.53874360634455
0_MFCC_8: 3.2812403479982555
0_Spectrogram mean coefficient_8.06Hz: 2.9304808810335685
0_MFCC_10: -2.3258697957092758
Processing Class: 1


100%|██████████| 40/40 [00:37<00:00,  1.06it/s]


Total Class 1 run time: 47.73957347869873
1
--------------------------------------------------
Top 5 Feature Changes:
0_LPCC_0: 3353.3718527151395
0_Spectrogram mean coefficient_41.94Hz: 531.8371739767856
0_Spectrogram mean coefficient_43.55Hz: 504.8919690484615
0_Spectrogram mean coefficient_45.16Hz: 435.84494044886725
0_Spectrogram mean coefficient_50.0Hz: 433.4383447545708
--------------------------------------------------
Top 5 Feature Changes:
0_Spectrogram mean coefficient_50.0Hz: 2137.5722572697337
0_LPCC_0: 1860.9818269615487
0_Spectrogram mean coefficient_41.94Hz: 608.333197316201
0_Spectrogram mean coefficient_46.77Hz: 550.9495042882082
0_Spectrogram mean coefficient_48.39Hz: 541.6613113291406
--------------------------------------------------
Top 5 Feature Changes:
0_LPCC_0: 3993.750832191493
0_Spectrogram mean coefficient_41.94Hz: 778.5364026426522
0_Spectrogram mean coefficient_43.55Hz: 627.1509462404066
0_Spectrogram mean coefficient_45.16Hz: 566.6336151739478
0_Spectrogr

...

100%|██████████| 6/6 [06:50<00:00, 68.44s/it]


Total Class 1 run time: 594.1245412826538
standing
--------------------------------------------------
Top 5 Feature Changes:
3_ECDF Percentile_1: 289504950.4086908
1_Median: -84554455.36501649
2_Median: 10198019.498019801
3_Median diff: 4554455.445544545
2_Median diff: -2376237.623762378
--------------------------------------------------
Top 5 Feature Changes:
3_Histogram mode: -171.57422311089772
1_Spectral decrease: 58.53884105482674
2_MFCC_6: 13.755271227123615
2_Max power spectrum: 12.755579895777968
1_MFCC_6: 10.795238372540583
avg_diff is None for rocket in standing. No correct model predictions!
Processing Class: 2


100%|██████████| 5/5 [05:42<00:00, 68.48s/it]


Total Class 2 run time: 485.2788972854614
walking
avg_diff is None for lstm in walking. No correct model predictions!
--------------------------------------------------
Top 5 Feature Changes:
3_Median diff: -32673267.32673256
3_Histogram mode: -48.534643758417666
1_MFCC_3: 14.581709450159654
2_MFCC_6: -11.38379851314951
3_Wavelet absolute mean_12.5Hz: 8.623027934284936
avg_diff is None for rocket in walking. No correct model predictions!
----------------------------------------------------------------------
Dataset Name: TwoLeadECG
--------------------------------------------------
X_train Shape: (929, 1, 82)
X_test Shape: (233, 1, 82)
y_train Counts: (array([0, 1], dtype=int64), array([466, 463], dtype=int64))
y_test Counts: (array([0, 1], dtype=int64), array([115, 118], dtype=int64))
--------------------------------------------------
{'lstm': 0.9914163090128756, 'catch22': 0.9699570815450643, 'rocket': 1.0}
--------------------------------------------------
Processing Class: 0


 67%|██████▋   | 157/233 [02:20<01:07,  1.12it/s]